In [68]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn.neighbors import BallTree

In [69]:
df = pd.read_csv(r"C:\Users\Jason\Downloads\CMPD_Incidents.csv")
df.head(10)

C:\Users\Jason\AppData\Local\Temp\ipykernel_6408\1129344954.py:1: DtypeWarning: Columns (0: ZIP) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Jason\Downloads\CMPD_Incidents.csv")


,X,Y,YEAR,INCIDENT_REPORT_ID,LOCATION,CITY,STATE,ZIP,X_COORD_PUBLIC,Y_COORD_PUBLIC,...,LOCATION_TYPE_DESCRIPTION,PLACE_TYPE_DESCRIPTION,PLACE_DETAIL_DESCRIPTION,CLEARANCE_STATUS,CLEARANCE_DETAIL_STATUS,CLEARANCE_DATE,HIGHEST_NIBRS_CODE,HIGHEST_NIBRS_DESCRIPTION,OBJECTID,GlobalID
0,1.466022e+06,594698.000101,2022,20220124-1339-00,9000 SANDERS CREEK CT,CHARLOTTE,NC,NaN,1466022,594698,...,Indoors,Residential,Private Residence,Exceptionally Cleared,Cleared by Other Means,2022/02/11 00:00:00+00,899,Other Unlisted Non-Criminal,1,{2BFC43E3-714C-497A-B4E4-965D93E3C92D}
1,1.471971e+06,556455.000144,2022,20221106-0405-50,5900 N TRYON ST,CHARLOTTE,NC,NaN,1471971,556455,...,Parking Lot,Commercial Place,Bar/Tavern/Nightclub,Exceptionally Cleared,Located (Missing Persons and Runaways only),2022/11/07 00:00:00+00,800,Missing Person,2,{50746513-1316-4FB6-9287-A87FD6A56148}
2,1.435269e+06,541933.999986,2019,20191226-1916-01,3200 WILKINSON BV,CHARLOTTE,NC,28208.0,1435269,541934,...,Indoors,Retail,Grocery Store/Supermarket,Open,Open,NaN,899,Other Unlisted Non-Criminal,3,{4E1032ED-2480-4FFB-B521-7D7E330D51CB}
3,1.439788e+06,547704.999927,2025,20251121-0626-01,1200 EFFINGHAM RD,CHARLOTTE,NC,28208.0,1439788,547705,...,Outdoors,Open Area,Street/Highway,Open,Open,NaN,290,Damage/Vandalism Of Property,4,{8B0D0A68-EE5A-4069-BD07-398DA9497DB7}
4,1.452009e+06,538012.999954,2024,20240721-0825-00,900 METROPOLITAN AV,CHARLOTTE,NC,28204.0,1452009,538013,...,Indoors,Retail,Department Store,Open,Open,NaN,23C,Shoplifting,5,{94318131-9153-4487-88D8-587380539A4D}
5,1.442855e+06,540024.999867,2021,20210321-1820-02,1700 MERRIMAN AV,CHARLOTTE,NC,NaN,1442855,540025,...,Indoors,Residential,Apartment/Duplex Private Res,Open,Open,NaN,13A,Aggravated Assault,6,{BCDEC76D-360E-4D8A-BFDC-E151E9D1AE22}
6,1.479393e+06,534984.000107,2025,20250207-1019-01,1300 KELSTON PL,CHARLOTTE,NC,28273.0,1479393,534984,...,Other,Open Area,Other - Open Area,Exceptionally Cleared,Victim Chose not to Prosecute,2025/09/23 00:00:00+00,13C,Intimidation,7,{7F1DC8B0-7141-4217-B0D3-FDF5BFBC0C1E}
7,1.483528e+06,514464.000136,2021,20210714-1452-03,8500 E INDEPENDENCE BV,CHARLOTTE,NC,28227.0,1483528,514464,...,Outdoors,Commercial Place,Hotel/Motel,Exceptionally Cleared,Located (Missing Persons and Runaways only),2021/07/16 00:00:00+00,800,Missing Person,8,{43FE2E58-D4F7-41F4-93FD-72958A762ADC}
8,1.453419e+06,514991.000066,2020,20200929-0857-00,4400 SHARON RD,CHARLOTTE,NC,28211.0,1453419,514991,...,Indoors,Retail,Shopping Mall,Open,Open,NaN,23H,All Other Thefts,9,{FA54D04C-D5FC-43A2-9409-8B845614F298}
9,1.416485e+06,573216.000086,2020,20200330-2012-00,3000 WESTWINDS CT,CHARLOTTE,NC,28216.0,1416485,573216,...,Indoors,Residential,Private Residence,Exceptionally Cleared,Victim Chose not to Prosecute,2020/04/17 00:00:00+00,13B,Simple Assault,10,{25E2CC1F-BE0D-47E0-A70B-974A4093EA66}


In [70]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 810660 entries, 0 to 810659
Data columns (total 29 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   X                          810660 non-null  float64
 1   Y                          810660 non-null  float64
 2   YEAR                       810660 non-null  int64  
 3   INCIDENT_REPORT_ID         810660 non-null  str    
 4   LOCATION                   810660 non-null  str    
 5   CITY                       810660 non-null  str    
 6   STATE                      810548 non-null  str    
 7   ZIP                        536445 non-null  object 
 8   X_COORD_PUBLIC             810660 non-null  int64  
 9   Y_COORD_PUBLIC             810660 non-null  int64  
 10  LATITUDE_PUBLIC            810660 non-null  float64
 11  LONGITUDE_PUBLIC           810660 non-null  float64
 12  DIVISION_ID                810660 non-null  str    
 13  CMPD_PATROL_DIVISION       809947 non-nu

In [71]:
df.isnull().sum()

X                                 0
Y                                 0
YEAR                              0
INCIDENT_REPORT_ID                0
LOCATION                          0
CITY                              0
STATE                           112
ZIP                          274215
X_COORD_PUBLIC                    0
Y_COORD_PUBLIC                    0
LATITUDE_PUBLIC                   0
LONGITUDE_PUBLIC                  0
DIVISION_ID                       0
CMPD_PATROL_DIVISION            713
NPA                               0
DATE_REPORTED                     0
DATE_INCIDENT_BEGAN               0
DATE_INCIDENT_END            192666
ADDRESS_DESCRIPTION               3
LOCATION_TYPE_DESCRIPTION         0
PLACE_TYPE_DESCRIPTION            0
PLACE_DETAIL_DESCRIPTION          0
CLEARANCE_STATUS                  0
CLEARANCE_DETAIL_STATUS           0
CLEARANCE_DATE               482845
HIGHEST_NIBRS_CODE                0
HIGHEST_NIBRS_DESCRIPTION         0
OBJECTID                    

In [72]:
df['CITY'].value_counts()

CITY
CHARLOTTE             808942
MECKLENBURG              596
MATTHEWS                 304
PINEVILLE                303
HUNTERSVILLE             185
                       ...  
CHARLOTTE/NC/28269         1
28273                      1
CHARLOTET                  1
CHARLOTTE-                 1
CHARLOTTE`                 1
Name: count, Length: 90, dtype: int64

In [73]:
df["ZIP"] = (
    df["ZIP"]
    .astype(str)
    .str.strip()
    .replace({
        "": pd.NA,
        "NA": pd.NA,
        "N/A": pd.NA,
        "NULL": pd.NA,
        "null": pd.NA,
        "None": pd.NA,
        "00000": pd.NA
    })
)

df["ZIP"] = df["ZIP"].str.extract(r"(\d{5})")[0]

vc = df["ZIP"].value_counts()

df = df[
    df["ZIP"].isna() |      # preserve missing
    (df["ZIP"].map(vc) >= 10)
]
df["ZIP"] = df["ZIP"].astype("category")

In [74]:
df['ZIP'].isna().sum()

np.int64(275133)

In [75]:
df['ZIP'].value_counts()

ZIP
28208    55984
28205    46235
28216    41528
28269    35952
28262    31032
         ...  
28083       10
28230       10
28231       10
28286       10
29203       10
Name: count, Length: 122, dtype: int64

In [76]:
# Missing Zip Code Imputation

known_zip = df[df["ZIP"].notna()].copy()
missing_zip = df[df["ZIP"].isna()].copy()
coords = known_zip[["LATITUDE_PUBLIC", "LONGITUDE_PUBLIC"]].to_numpy()

eps = 0.5 / 6371

db = DBSCAN(
    eps=eps,
    min_samples=20,
    metric="haversine"
)

known_zip["cluster"] = db.fit_predict(np.radians(coords))
known_zip = known_zip[known_zip["cluster"] != -1]

cluster_zip_map = (
    known_zip
    .groupby("cluster")["ZIP"]
    .agg(lambda x: x.mode().iloc[0])
)

centroids = (
    known_zip
    .groupby("cluster")[["LATITUDE_PUBLIC", "LONGITUDE_PUBLIC"]]
    .mean()
)

tree = BallTree(
    np.radians(centroids.to_numpy()),
    metric="haversine"
)

dist, idx = tree.query(
    np.radians(missing_zip[["LATITUDE_PUBLIC", "LONGITUDE_PUBLIC"]].to_numpy()),
    k=1
)

# ~1km cutoff
missing_zip["dist_km"] = dist.flatten() * 6371
missing_zip["cluster"] = centroids.index[idx.flatten()]

missing_zip["ZIP"] = missing_zip["cluster"].map(cluster_zip_map)

df = pd.concat([
    known_zip.drop(columns="cluster"),
    missing_zip.drop(columns=["cluster", "dist_km"])
])

In [77]:
df['ZIP'].value_counts()

ZIP
28208    186659
28269     78011
28273     59013
28227     51039
28217     49254
          ...  
28286        10
29203        10
28031         5
28036         2
28107         2
Name: count, Length: 122, dtype: int64

In [78]:
df.info()

<class 'pandas.DataFrame'>
Index: 809058 entries, 2 to 810655
Data columns (total 29 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   X                          809058 non-null  float64 
 1   Y                          809058 non-null  float64 
 2   YEAR                       809058 non-null  int64   
 3   INCIDENT_REPORT_ID         809058 non-null  str     
 4   LOCATION                   809058 non-null  str     
 5   CITY                       809058 non-null  str     
 6   STATE                      808946 non-null  str     
 7   ZIP                        809058 non-null  category
 8   X_COORD_PUBLIC             809058 non-null  int64   
 9   Y_COORD_PUBLIC             809058 non-null  int64   
 10  LATITUDE_PUBLIC            809058 non-null  float64 
 11  LONGITUDE_PUBLIC           809058 non-null  float64 
 12  DIVISION_ID                809058 non-null  str     
 13  CMPD_PATROL_DIVISION       808

In [79]:
df['HIGHEST_NIBRS_DESCRIPTION'].value_counts()

HIGHEST_NIBRS_DESCRIPTION
Theft From Motor Vehicle       81771
All Other Offenses             78775
Other Unlisted Non-Criminal    76107
Simple Assault                 69655
All Other Thefts               69281
                               ...  
Welfare Fraud                      9
Betting/Wagering                   9
Purchasing Prostitution            4
Bribery                            3
Gas Leak                           1
Name: count, Length: 72, dtype: int64

In [80]:
df['STATE'].value_counts()

STATE
NC    808937
ND         5
IN         2
SC         2
Name: count, dtype: int64

In [81]:
df['LOCATION_TYPE_DESCRIPTION'].value_counts()

LOCATION_TYPE_DESCRIPTION
Indoors         354614
Outdoors        291142
Parking Lot     126331
Other            24096
Parking Deck     12875
Name: count, dtype: int64

In [82]:
df['PLACE_TYPE_DESCRIPTION'].value_counts()

PLACE_TYPE_DESCRIPTION
Residential               377236
Open Area                 129324
Commercial Place          129243
Retail                    123075
Public/Non-Residential     50180
Name: count, dtype: int64

In [83]:
df['CLEARANCE_STATUS'].value_counts()

CLEARANCE_STATUS
Open                                   488917
Exceptionally Cleared                  158591
Cleared by Arrest                      127251
Unfounded                               27272
Cleared by Arrest by Another Agency      7027
Name: count, dtype: int64

In [84]:
df['YEAR'].value_counts()

YEAR
2023    95342
2024    92106
2019    92011
2025    90962
2017    90131
2022    89397
2018    88256
2021    84791
2020    83496
2026     2566
Name: count, dtype: int64

In [85]:
df_clean = df.drop(columns=['GlobalID', 'X', 'Y', 'STATE', 'X_COORD_PUBLIC', 'Y_COORD_PUBLIC', 'DIVISION_ID', 'CLEARANCE_DATE', 'DATE_INCIDENT_END', 'CITY', 'STATE', 'ADDRESS_DESCRIPTION', 'HIGHEST_NIBRS_CODE', 'CLEARANCE_STATUS', 'CLEARANCE_DETAIL_STATUS', 'INCIDENT_REPORT_ID', 'OBJECTID', 'DATE_REPORTED', 'YEAR'])

In [86]:
df_clean['CMPD_PATROL_DIVISION'].fillna('Unknown', inplace=True)

C:\Users\Jason\AppData\Local\Temp\ipykernel_6408\1276217217.py:1: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  df_clean['CMPD_PATROL_DIVISION'].fillna('Unknown', inplace=True)


2               Freedom
3                 Metro
4               Central
6         Hickory Grove
7          Independence
              ...      
810648         Westover
810649            Metro
810650            North
810653            South
810655     Independence
Name: CMPD_PATROL_DIVISION, Length: 809058, dtype: str

In [87]:
df_clean.isna().sum()

LOCATION                       0
ZIP                            0
LATITUDE_PUBLIC                0
LONGITUDE_PUBLIC               0
CMPD_PATROL_DIVISION         538
NPA                            0
DATE_INCIDENT_BEGAN            0
LOCATION_TYPE_DESCRIPTION      0
PLACE_TYPE_DESCRIPTION         0
PLACE_DETAIL_DESCRIPTION       0
HIGHEST_NIBRS_DESCRIPTION      0
dtype: int64

In [88]:
df_clean.info()

<class 'pandas.DataFrame'>
Index: 809058 entries, 2 to 810655
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   LOCATION                   809058 non-null  str     
 1   ZIP                        809058 non-null  category
 2   LATITUDE_PUBLIC            809058 non-null  float64 
 3   LONGITUDE_PUBLIC           809058 non-null  float64 
 4   CMPD_PATROL_DIVISION       808520 non-null  str     
 5   NPA                        809058 non-null  int64   
 6   DATE_INCIDENT_BEGAN        809058 non-null  str     
 7   LOCATION_TYPE_DESCRIPTION  809058 non-null  str     
 8   PLACE_TYPE_DESCRIPTION     809058 non-null  str     
 9   PLACE_DETAIL_DESCRIPTION   809058 non-null  str     
 10  HIGHEST_NIBRS_DESCRIPTION  809058 non-null  str     
dtypes: category(1), float64(2), int64(1), str(7)
memory usage: 151.9 MB


In [89]:
df_clean.to_csv('data\cmpd_cleaned.csv', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Jason\AppData\Local\Temp\ipykernel_6408\1243101713.py:1: SyntaxWarning: invalid escape sequence '\c'
  df_clean.to_csv('data\cmpd_cleaned.csv', index=False)
